In [113]:
// SETUP
use std::fs::File;
use std::collections::{HashMap, HashSet};
use std::str::FromStr;
use std::io::{BufRead, BufReader, Error, ErrorKind, Read};
use std::time::Instant;

fn read_lines(path: &str) -> Result<Vec<String>, Error> {
    let file = File::open(path)?; 
    BufReader::new(file).lines().collect()
}

fn read_integers(path: &str) -> Result<Vec<i64>, Error> {
    let mut v = Vec::new();
    for line in read_lines(path)? {
        let n = line   
            .trim() 
            .parse() 
            .map_err(|e| Error::new(ErrorKind::InvalidData, e))?; 
        v.push(n);
    }
    Ok(v)
}

const DATA_ROOT: &str = "data/";

fn data_file(problem_id: u32) -> String {
    format!("{}/{:02}.txt", DATA_ROOT, problem_id)
}

fn test_file(problem_id: u32, test_id: u32) -> String {
    format!("{}/{:02}t{}.txt", DATA_ROOT, problem_id, test_id)
}

fn extra_file(problem_id: u32, suffix: &str) -> String {
    format!("{}/{:02}.{}.txt", DATA_ROOT, problem_id, suffix)
}

In [4]:
// DAY01
const TOTAL: i64 = 2020;

fn problem01() {    
    let nums: Vec<i64> = read_integers(&data_file(1)).unwrap();    
    let mut reg = HashMap::new();

    for (i, n) in nums.iter().enumerate() {
        let rest = TOTAL - n;
        if reg.contains_key(&rest) {
            println!("{} * {} = {}", n, rest, n * rest);
        }
        reg.insert(n, i);
    }
    
    for (i, n1) in nums.iter().enumerate() {
        for j in (i + 1)..nums.len() {
            let n2 = nums[j];
            let rest = TOTAL - n1 - n2;
            match reg.get(&rest) {
                Some(k) if k > &j => {
                    println!("{} * {} * {} = {}", n1, n2, rest, n1 * n2 * rest)
                },
                _ => (),
            }
        }
    }
}

problem01();

1564 * 456 = 713184
764 * 857 * 399 = 261244452


In [50]:
// DAY02
#[derive(Debug, PartialEq)]
struct PasswordDesc {
    cnt1: usize,
    cnt2: usize,
    ch: char,
    pwd: String,
}

impl FromStr for PasswordDesc {
    type Err = std::num::ParseIntError;
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let parts: Vec<&str> = s.split(' ').collect();
        let counts: Vec<usize> = parts[0]
            .split('-')
            .map(|p| p.parse().unwrap())
            .collect();
        Ok(PasswordDesc { 
            cnt1: counts[0], 
            cnt2: counts[1], 
            ch: parts[1].as_bytes()[0] as char, 
            pwd: parts[2].to_owned()})
    }
}

fn is_valid1(p: &PasswordDesc) -> bool {
    let cnt = p.pwd.matches(p.ch).count();
    p.cnt1 <= cnt && cnt <= p.cnt2
}

fn is_valid2(p: &PasswordDesc) -> bool {
    let pwd = p.pwd.as_bytes();
    let ch = p.ch as u8;
    (pwd[p.cnt1 - 1] == ch) ^ (pwd[p.cnt2 - 1] == ch)
}

fn problem02() {
    let passwords: Vec<PasswordDesc> = 
        read_lines(&data_file(2)).unwrap()
        .iter()
        .filter_map(|line| line.parse().ok())
        .collect();

    println!("Number of valid passwords: {}, {}", 
        passwords.iter().filter(|x| is_valid1(x)).count(), 
        passwords.iter().filter(|x| is_valid2(x)).count());
}

problem02();

Number of valid passwords: 548, 502


In [72]:
// DAY03
const TREE: u8 = '#' as u8;

fn count_trees(rows: &Vec<String>, dx: usize, dy: usize) -> usize {
    let h = rows.len();
    if h == 0 {
        return 0;
    }
    let w = rows[0].len();
    let mut x: usize = 0;
    let mut res: usize = 0;
    for y in (0..h).step_by(dy) {
        res += (rows[y].as_bytes()[x % w] == TREE) as usize;
        x += dx;
    }
    res
}

fn problem03() {
    let rows = read_lines(&data_file(3)).unwrap();
    let res1 = count_trees(&rows, 3, 1);
    println!("Num trees 1: {}", res1);
    
    let res2: usize = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
        .iter()
        .map(|(dx, dy)| count_trees(&rows, *dx, *dy))
        .product();
    println!("Num trees 2: {}", res2);
}

problem03();

Num trees 1: 242
Num trees 2: 2265549792


In [93]:
// DAY04
#[macro_use] extern crate lazy_static;
extern crate regex;

use regex::Regex;
use std::fs;
use std::collections::HashSet;

type Passport = HashMap<String, String>;

fn read_passport(line: &str) -> Passport {
    line.split_whitespace()
        .filter_map(|s| {
            let mut parts = s.split(':');
            Some((parts.next()?.to_owned(), parts.next()?.to_owned())) 
        })
        .collect()
}

const REQUIRED_FIELDS: [&str; 7] = ["byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"];

fn is_valid1(passport: &Passport) -> bool {
    REQUIRED_FIELDS.iter().all(|f| passport.contains_key(*f))    
}

fn is_valid2(passport: &Passport) -> bool {
    lazy_static! {
        static ref PID_REGEX: Regex = Regex::new(r"^(\d{9})$").unwrap();
        static ref HCL_REGEX: Regex = Regex::new(r"^#[0-9a-f]{6}$").unwrap();
        static ref HGT_REGEX: Regex = Regex::new(r"^(?P<in>\d+)in|(?P<cm>\d+)cm$").unwrap();
        static ref EYE_COLORS: HashSet<String> = ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
            .iter()
            .copied()
            .map(|x| x.to_owned())
            .collect();
    }
    
    fn in_range<T: Ord + FromStr>(int_str: &str, minv: T, maxv: T) -> bool {
        match int_str.parse::<T>() {
            Ok(n) => n >= minv && n <= maxv,
            _ => false
        }
    }
    
    fn is_valid_height(height_str: &str) -> bool {
        match HGT_REGEX.captures(height_str) {
            Some(c) => 
                match (c.name("in"), c.name("cm")) {
                    (Some(x), _) => in_range(x.as_str(), 59, 76),
                    (_, Some(x)) => in_range(x.as_str(), 150, 193),
                    _ => false,
                },
            _ => false
        }
    }
    
    is_valid1(passport) &&
    in_range(&passport["byr"], 1920, 2002) &&
    in_range(&passport["iyr"], 2010, 2020) &&
    in_range(&passport["eyr"], 2020, 2030) &&
    EYE_COLORS.contains(passport["ecl"].as_str()) &&
    PID_REGEX.is_match(&passport["pid"]) &&
    HCL_REGEX.is_match(&passport["hcl"]) &&
    is_valid_height(&passport["hgt"])
}

fn problem04() {
    let passports: Vec<Passport> = fs::read_to_string(&data_file(4))
        .unwrap()
        .split("\n\n")
        .map(|s| read_passport(s))
        .collect();

    println!("Valid passports 1: {}", passports.iter().filter(|x| is_valid1(x)).count());
    println!("Valid passports 2: {}", passports.iter().filter(|x| is_valid2(x)).count());
}

problem04();

Valid passports 1: 264
Valid passports 2: 224


In [50]:
// DAY05
use std::collections::HashSet;
use std::iter::FromIterator;

fn get_id(seat: &str) -> u32 {
    seat.chars().fold(0, |res, c| res * 2 + "BR".contains(c) as u32)
}

fn problem05() {
    let lines = read_lines(&data_file(5)).unwrap();
    let ids: Vec<u32> = lines
        .iter()
        .map(|x| get_id(x))
        .collect();
    
    let max_id: u32 = *ids.iter().max().unwrap();
    let id_set = ids.iter().cloned().collect::<HashSet<u32>>();
    
    let mut seat: u32 = 0;
    for i in 0..=(1 << lines[0].len()) {
        let id = i as u32;
        if !id_set.contains(&id) && 
            id_set.contains(&(id + 1)) && 
            id_set.contains(&(id - 1)) {
            seat = id;
            break;
        }
    }
    println!("Max ID: {}, Seat: {}", max_id, seat);
}

problem05();

Max ID: 801, Seat: 597


In [44]:
// DAY06
use std::fs;
use std::collections::HashMap;

fn problem06() {
    let text = fs::read_to_string(&data_file(6)).unwrap();
    let groups: Vec<Vec<&str>> = text
            .split("\n\n")
            .map(|s| s.split_whitespace().collect())
            .collect();
    
    let (mut res1, mut res2) = (0, 0);
    for group in &groups {
        let mut counts: HashMap<char, usize> = HashMap::new();
        for answers in group {
            for ans in answers.chars() {
                *counts.entry(ans).or_insert(0) += 1;
            }
        }
        res1 += counts.len();
        res2 += counts.iter().filter(|&(_, v)| *v == group.len()).count()
    }
    println!("Answer 1: {}, Answer 2: {}", res1, res2);
}

problem06();

Answer 1: 6310, Answer 2: 3193


In [63]:
// DAY07
type Mapping = HashMap<String, HashMap<String, u32>>;
type BackMapping = HashMap<String, Vec<String>> ;

fn parse_mapping(line: &str) -> (String, HashMap<String, u32>) {
    let parts: Vec<&str> = line.split_whitespace().collect();
    let bag_color = parts[0..=1].join(" ");
    let mut res: HashMap<String, u32> = HashMap::new();
    for i in (4..parts.len()).step_by(4) {
        if parts[i] != "no" {
            let child_bag_color = parts[i + 1..i + 3].join(" ");
            res.insert(child_bag_color, parts[i].parse::<u32>().unwrap());
        }
    }
    (bag_color, res)
}

fn build_back_mapping(mapping: &Mapping) -> BackMapping {
    let mut res: HashMap<_, _> = HashMap::new();
    for (key, children) in mapping {
        for (child, _) in children {
            res.entry(child.clone()).or_insert(Vec::new()).push(key.clone());
        }
    }
    res
}

fn count_reachable(root: &str, back_mapping: &BackMapping) -> u32 {
    fn rec(node: &str, back_mapping: &BackMapping, visited: &mut HashSet<String>) -> u32 {
        if visited.contains(node) {
            return 0;
        }
        let mut res = 1;
        if back_mapping.contains_key(node) {
            for child in &back_mapping[node] {
                res += rec(&child, back_mapping, visited);
            }
        }
        visited.insert(node.to_owned());
        res
    }
    let mut visited: HashSet<String> = HashSet::new();
    rec(root, back_mapping, &mut visited)
}

fn count_contains(root: &str, mapping: &Mapping) -> u32 {
    let mut res: u32 = 1;
    for (child, count) in &mapping[root] {
        res += count * count_contains(&child, mapping);
    }
    res
}

fn problem07() {
    let mapping: Mapping = read_lines(&data_file(7))
        .unwrap()
        .iter()
        .map(|line| parse_mapping(line))
        .into_iter()
        .collect();
    let back_mapping = build_back_mapping(&mapping);
    const ROOT: &str = "shiny gold";
    println!("Reachable: {}", count_reachable(ROOT, &back_mapping) - 1);
    println!("Contains: {}", count_contains(ROOT, &mapping) - 1);
}

problem07();

Reachable: 370
Contains: 29547


In [43]:
// DAY08

#[derive(Debug, PartialEq, Copy, Clone)]
enum Op {
    Acc(i64),
    Nop(i64),
    Jmp(i64),
}

impl FromStr for Op {
    type Err = ();
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let mut it = s.split(' ');
        let opstr: &str = it.next().ok_or(())?;
        let val: i64 = it.next().ok_or(())?.parse::<i64>().map_err(|_|())?;
        match opstr {
            "acc"  => Ok(Op::Acc(val)),
            "nop"  => Ok(Op::Nop(val)),
            "jmp"  => Ok(Op::Jmp(val)),
            _      => Err(()),
        }
    }
}

fn eval_ops(ops: &Vec<Op>) -> (bool, i64) {
    let n = ops.len() as i64;
    let mut visited: Vec<bool> = Vec::new();
    visited.resize(ops.len(), false);
    let mut ip: i64 = 0;
    let mut acc: i64 = 0;
    while ip >= 0 && ip < n {
        if visited[ip as usize] {
            return (false, acc)
        }
        visited[ip as usize] = true;
        match ops[ip as usize] {
            Op::Jmp(val) => ip += val - 1,
            Op::Acc(val) => acc += val,
            _ => (),
        }
        ip += 1;
    }
    (true, acc)
}

fn try_mutate_program(ops: &Vec<Op>) -> Option<i64> {
    let mut ops_copy = ops.clone();
    for (i, op) in ops.iter().enumerate() {
        let prevOp: Op = *op;
        let newOp = match op {
            Op::Jmp(val) => Op::Nop(*val),
            Op::Nop(val) => Op::Jmp(*val),
            _ => prevOp
        };
        if prevOp != newOp {
            ops_copy[i] = newOp;
            let (terminated, acc) = eval_ops(&ops_copy);
            if terminated {
                return Some(acc);
            }
            ops_copy[i] = prevOp;
        }
    }
    None
}

fn problem08() {
    let ops: Vec<Op> = 
        read_lines(&data_file(8)).unwrap()
        .iter()
        .filter_map(|line| line.parse().ok())
        .collect();
    let (_, acc) = eval_ops(&ops);
    println!("Answer 1: {}", acc);
    println!("Answer 2: {}", try_mutate_program(&ops).unwrap());
}

problem08();

Answer 1: 1782
Answer 2: 797


In [111]:
// DAY09
use std::collections::BTreeMap;

fn get_first_non_sum(nums: &Vec<i64>, k: usize) -> Option<i64> {
    // use btree data structure to maintain a sorted window of size k
    let mut window = BTreeMap::new();
    let n = nums.len();
    if n < k {
        return None;
    }
    for i in 0..k {
        *window.entry(nums[i]).or_insert(1) += 1;
    }
    for i in k..n {
        let s = nums[i];
        let (mut it_l, mut it_r) = (window.iter(), window.iter().rev());
        let (mut l, mut r) = (it_l.next()?, it_r.next()?);
        while l.0 != r.0 {
            let cur_sum = l.0 + r.0;
            if cur_sum < s {
                l = it_l.next()?;
            } else if cur_sum > s {
                r = it_r.next()?;
            } else {
                break;
            }
        }
        if l.0 == r.0 && *l.1 == 1 {
            return Some(s)
        }
        
        let to_remove = nums[i - k];
        window.entry(to_remove).and_modify(|e| { *e -= 1 });
        if window[&to_remove] == 0 {
            window.remove(&to_remove);
        }
        *window.entry(s).or_insert(1) += 1;
    }
    None
}

fn find_sum_range(nums: &Vec<i64>, s: i64) -> Option<(usize, usize)>{
    let n = nums.len();
    if n == 0 {
        return None
    }
    let (mut l, mut r) = (0, 0);
    let mut cur_sum = nums[0];
    loop {
        if cur_sum < s {
            r += 1;
            cur_sum += nums[r];
        } else if cur_sum > s {
            cur_sum -= nums[l];
            l += 1;
        } else {
            break;
        }
        if r == n || l == n {
            break;
        }
    }
    if l != r {
        Some((l, r))
    } else {
        None
    }
}

fn find_sum_range_agg(nums: &Vec<i64>, s: i64) -> Option<i64>{
    let range = find_sum_range(&nums, s)?;
    let slice = &nums[range.0..=range.1];
    let max = slice.iter().max()?;
    let min = slice.iter().min()?;    
    Some(min + max)
}

fn problem09() {
    const WINDOW_SIZE: usize = 25;
    let nums: Vec<i64> = read_integers(&data_file(9)).unwrap();
    let non_sum = get_first_non_sum(&nums, WINDOW_SIZE).unwrap();
    println!("First non-sum: {}", non_sum);
    println!("Sum range: {}", find_sum_range_agg(&nums, non_sum).unwrap());
}

problem09();

First non-sum: 85848519
Sum range: 13414198


In [131]:
// DAY10

fn problem10() {
    let mut nums: Vec<i64> = read_integers(&data_file(10)).unwrap();
    nums.sort();
    nums.insert(0, 0);
    nums.push(nums.iter().max().unwrap() + 3);
    
    let n = nums.len();
    let mut diffs: Vec<i64> = Vec::new();
    for i in 1..n {
        diffs.push(nums[i] - nums[i - 1]);
    }
    
    let diffs1: i64 = diffs.iter().map(|x| (*x == 1) as i64).sum();
    let diffs3: i64 = diffs.iter().map(|x| (*x == 3) as i64).sum();
    
    println!("Sum 1/3 diffs: {}", diffs1 * diffs3);
    
    fn count_arrangements(nums: &Vec<i64>) -> i64 {
        fn rec(nums: &Vec<i64>, idx: usize, counts: &mut Vec<i64>) -> i64 {
            let n = nums.len();
            if idx == n - 1 {
                return 1;
            }
            if counts[idx] >= 0 {
                return counts[idx];
            }
            let mut res = 0;
            let mut i = idx;
            while i < n - 1 {
                i += 1;
                let diff = nums[i] - nums[idx];
                if diff <= 3 {
                    res += rec(nums, i, counts);
                } else {
                    break;
                }
            }
            counts[idx] = res;
            res
        }
        let mut counts: Vec<i64> = Vec::new();
        counts.resize(nums.len(), -1);
        rec(nums, 0, &mut counts)
    }
    
    println!("Number of arrangements: {}", count_arrangements(&nums));
}

problem10();

Sum 1/3 diffs: 2414
Number of arrangements: 21156911906816


In [38]:
// DAY11
use std::fmt;

#[derive(Debug, PartialEq, Copy, Clone)]
enum Seat {
    Floor,
    Vacant,
    Busy,
}

type SeatPlan = Vec<Vec<Seat>>;

impl FromStr for Seat {
    type Err = ();
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        match s {
            "."  => Ok(Seat::Floor),
            "L"  => Ok(Seat::Vacant),
            "#"  => Ok(Seat::Busy),
            _      => Err(()),
        }
    }
}

const OFFS: [(i32, i32); 8] = [
    (-1, -1), (0, -1), (1, -1), 
    (-1, 0), (1, 0),
    (-1, 1), (0, 1), (1, 1)];

fn get_seat(seats: &SeatPlan, (x, y): (i32, i32)) -> Option<Seat> {
    let (h, w) = (seats.len() as i32, seats[0].len() as i32);
    if x >= 0 && x < w && y >= 0 && y < h {
        Some(seats[y as usize][x as usize])
    } else {
        None
    }
}

fn get_neighbor_counts(seats: &SeatPlan) -> Vec<Vec<u32>> {
    let (h, w) = (seats.len() as i32, seats[0].len() as i32);
    let mut res = vec![vec![0; w as usize]; h as usize];
    for y in 0..h {
        for x in 0..w {
            res[y as usize][x as usize] = OFFS
                .iter()
                .filter_map(|(dx, dy)| get_seat(&seats, (x + dx, y + dy)))
                .map(|s| (s == Seat::Busy) as u32)
                .sum();
        }
    }
    res
}

fn get_visible_neighbor_counts(seats: &SeatPlan) -> Vec<Vec<u32>> {
    let (h, w) = (seats.len() as usize, seats[0].len() as usize);
    let mut res = vec![vec![0; w]; h];
    let mut can_see = vec![vec![false; w]; h];
    
    for (dx, dy) in OFFS.iter() {
        for j in 0..h {
            for i in 0..w {
                let x: usize = if *dx <= 0 {i} else {w - i - 1};
                let y: usize = if *dy <= 0 {j} else {h - j - 1};
                let pos = (x as i32 + dx, y as i32 + dy);
                let visible = match get_seat(&seats, pos) {
                    Some(Seat::Floor) => can_see[pos.1 as usize][pos.0 as usize],
                    Some(Seat::Busy) => true,
                    Some(Seat::Vacant) => false,
                    _ => false
                };
                can_see[y][x] = visible;
                res[y][x] += visible as u32;
            }
        }
    }
    res
}

fn step(seats: &SeatPlan, 
        new_seats:&mut SeatPlan, 
        neighbors_fn: fn(&SeatPlan) -> Vec<Vec<u32>>, 
        max_occupied: u32) -> (u32, u32) {
    let (h, w) = (seats.len() as usize, seats[0].len() as usize);
    let neighbor_counts = neighbors_fn(seats);
    let mut num_changed = 0;
    let mut total_occupied = 0;
    for y in 0..h {
        for x in 0..w {
            let n = neighbor_counts[y][x];
            let status = match seats[y][x] {
                Seat::Vacant if n == 0 => Seat::Busy,
                Seat::Busy if n >= max_occupied => Seat::Vacant,
                val => val
            };
            new_seats[y][x] = status;
            num_changed += (status != seats[y][x]) as u32;
            total_occupied += (status == Seat::Busy) as u32;
        }
    }
    (num_changed, total_occupied)
}

fn iter_seats(seats: &SeatPlan, 
    neighbors_fn: fn(&SeatPlan) -> Vec<Vec<u32>>, 
    max_occupied: u32) -> u32 {
    let mut s1 = seats.clone();
    let mut s2 = seats.clone();
    let mut flip = true;
    loop {
        let (r1, r2) = if flip {(&s1, &mut s2)} else {(&s2, &mut s1)};
        let (changed, occupied) = step(r1, r2, neighbors_fn, max_occupied);
        if changed == 0 {
            return occupied;
        }
        flip = !flip;
    }
}

fn problem11() {
    let seats: SeatPlan = read_lines(&data_file(11))
        .unwrap()
        .iter()
        .map(|line| line.chars().map(|c| c.to_string().parse().unwrap()).collect())
        .collect();
    println!("Answer 1: {}", iter_seats(&seats, get_neighbor_counts, 4));
    println!("Answer 2: {}", iter_seats(&seats, get_visible_neighbor_counts, 5));
    
}

problem11();

Answer 1: 2211
Answer 2: 1995


In [40]:
// DAY12
#[derive(Debug, PartialEq, Copy, Clone)]
enum Action {
    North(i32),
    South(i32),
    East(i32),
    West(i32),
    Forward(i32),
    Left(i32),
    Right(i32),
}

impl FromStr for Action {
    type Err = ();
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let c = s.as_bytes()[0] as char;
        let val = s[1..].parse::<i32>().map_err(|_|())?;      
        match c {
            'N' => Ok(Action::North(val)),
            'S' => Ok(Action::South(val)),
            'E' => Ok(Action::East(val)),
            'W' => Ok(Action::West(val)),
            'F' => Ok(Action::Forward(val)),
            'L' => Ok(Action::Left(val)),
            'R' => Ok(Action::Right(val)),
            _      => Err(()),
        }
    }
}

fn manhattan((x, y): (i32, i32)) -> i32 {
    x.abs() + y.abs()
}

fn move_dir((x, y): (i32, i32), dir: i32, val: i32) -> (i32, i32) {
    let ang = (dir as f32).to_radians();
    let ca = ang.cos().round() as i32;
    let sa = ang.sin().round() as i32;
    (x + val * ca, y + val * sa)
}

fn step1((x, y, dir): (i32, i32, i32), cmd: &Action) -> (i32, i32, i32) {
    match cmd {
        Action::North(val) => (x, y - val, dir),
        Action::South(val) => (x, y + val, dir),
        Action::West(val) => (x - val, y, dir),
        Action::East(val) => (x + val, y, dir),
        Action::Right(val) => (x, y, (dir + val) % 360),
        Action::Left(val) => (x, y, (dir - val + 360) % 360),
        Action::Forward(val) => {
            let (x1, y1) = move_dir((x, y), dir, *val);
            (x1, y1, dir)
        }
    }
}

fn rotate((x, y): (i32, i32), angle: i32) -> (i32, i32) {
    let ar = (angle as f32).to_radians();
    let (ca, sa) = (ar.cos(), ar.sin());
    let (xf, yf)  = (x as f32, y as f32);
    let x1 = (ca * xf - sa * yf).round() as i32;
    let y1 = (sa * xf + ca * yf).round() as i32;
    (x1, y1)
}

fn step2((x, y, wx, wy): (i32, i32, i32, i32), cmd: &Action) -> (i32, i32, i32, i32) {
    match cmd {
        Action::North(val) => (x, y, wx, wy - val),
        Action::South(val) => (x, y, wx, wy + val),
        Action::West(val) => (x, y, wx - val, wy),
        Action::East(val) => (x, y, wx + val, wy),
        Action::Right(val) => {
            let (wx1, wy1) = rotate((wx, wy), *val);
            (x, y, wx1, wy1)
        }
        Action::Left(val) => {
            let (wx1, wy1) = rotate((wx, wy), -*val);
            (x, y, wx1, wy1)
        }
        Action::Forward(val) => (x + wx * val, y + wy * val, wx, wy)
    }
}

fn problem12() {
    let commands: Vec<Action> = read_lines(&data_file(12))
        .unwrap()
        .iter()
        .map(|line| line.parse().unwrap())
        .collect();
    
    let (x1, y1, angle1) = commands.iter().fold((0, 0, 0), |acc, cmd| step1(acc, cmd));
    println!("Answer 1: {}", manhattan((x1, y1)));
    
    let (x2, y2, wx2, wy2) = commands.iter().fold((0, 0, 10, -1), |acc, cmd| step2(acc, cmd));
    println!("Answer 2: {}", manhattan((x2, y2)));
}

problem12();

Answer 1: 508
Answer 2: 30761


In [3]:
// DAY13
fn part1(periods: &Vec<(u64, u64)>, ts: u64) -> u64 {
    let mut min_diff = None;
    let mut res = 0;
    for (p, i) in periods {
        let d = p - ts % p;
        if min_diff == None || d < min_diff.unwrap() {
            min_diff = Some(d);
            res = d * p;
        }
    }
    res
}

fn part2(periods: &Vec<(u64, u64)>) -> u64 {
    let (mut p0, i0) = periods[0];
    // Assuming here that the largest number has an offset 
    // which is present as another number and they are also coprime.
    // This is something I noticed for my data, but should be safe for any input.
    p0 *= i0; 
    let mut k = 1;
    loop {
        let ts = k * p0 - i0;
        let mut found = true;
        for j in 1..periods.len() {
            let (p, i) = periods[j];
            if (ts + i) % p != 0 {
                found = false;
                break;
            }
        }
        if found {
            return ts;
        }
        k += 1;
        if k % 1000000000 == 0 {
            println!("Processed timestamp: {}", ts);
        }
    }
}

fn problem13() {
    let now = Instant::now();

    let lines = read_lines(&data_file(13)).unwrap();
    let ts: u64 = lines[0].parse::<u64>().unwrap();
    let mut periods: Vec<(u64, u64)> = lines[1].split(",")
        .map(|p| p.parse::<u64>())
        .enumerate()
        .filter_map(|(i, p)| match p {
            Ok(val) => Some((val, i as u64)),
            _ => None
        })
        .collect();
    periods.sort_by(|a, b| b.0.partial_cmp(&a.0).unwrap());
    
    println!("Answer 1: {:?}", part1(&periods, ts));
    println!("Answer 2: {:?}", part2(&periods));

    let elapsed = now.elapsed();
    println!("Elapsed: {:.2?}", elapsed);
}

problem13();

Answer 1: 153
Processed timestamp: 12960999987026
Processed timestamp: 25921999987026
Processed timestamp: 38882999987026
Processed timestamp: 51843999987026
Processed timestamp: 64804999987026
Processed timestamp: 77765999987026
Processed timestamp: 90726999987026
Processed timestamp: 103687999987026
Processed timestamp: 116648999987026
Processed timestamp: 129609999987026
Processed timestamp: 142570999987026
Processed timestamp: 155531999987026
Processed timestamp: 168492999987026
Processed timestamp: 181453999987026
Processed timestamp: 194414999987026
Processed timestamp: 207375999987026
Processed timestamp: 220336999987026
Processed timestamp: 233297999987026
Processed timestamp: 246258999987026
Processed timestamp: 259219999987026
Processed timestamp: 272180999987026
Processed timestamp: 285141999987026
Processed timestamp: 298102999987026
Processed timestamp: 311063999987026
Processed timestamp: 324024999987026
Processed timestamp: 336985999987026
Processed timestamp: 3499469999

In [90]:
// DAY14
#[macro_use] extern crate lazy_static;
extern crate regex;

use regex::Regex;

#[derive(Debug, PartialEq)]
struct MemOp {
    mask: String,
    ops: Vec<(i64, i64)>,
}

fn parse_ops(data: &str) -> Vec<MemOp> {
    lazy_static! {
        static ref MEM_REGEX: Regex = 
            Regex::new(r"^mem\[(?P<addr>\d+)\] = (?P<value>\d+)$").unwrap();
    }
    
    fn get_match_val(cap: &regex::Captures, group_name: &str) -> Option<i64> {
        match cap.name(group_name)?.as_str().parse::<i64>() {
            Ok(val) => Some(val),
            _ => None
        }
    }
    
    fn parse_op(lines: &Vec<&str>) -> Option<MemOp> {
        if lines.len() <= 1 {
            return None;
        }
        Some(MemOp {
            mask: lines[0].to_owned(), 
            ops: lines[1..]
                .iter()
                .filter_map(|line| {
                    let c = MEM_REGEX.captures(line)?;
                    Some((get_match_val(&c, "addr")?, get_match_val(&c, "value")?))
                })
                .collect()
        })
    }
    
    data.split("mask = ")
        .filter_map(|chunk| parse_op(&chunk.split("\n").collect()))
        .collect()
}

fn apply_mask1(val: &i64, mask: &str) -> i64 {
    let mut res = *val;
    let mut bit = 1;
    for i in (0..mask.len()).rev() {
        match mask.as_bytes()[i] as char {
            '0' => res = res & !bit,
            '1' => res = res | bit,
            _ => (),
        };            
        bit = bit << 1;
    }
    res
}

fn eval1(mem_ops: &Vec<MemOp>) -> HashMap<i64, i64> {
    let mut mem = HashMap::new();
    for mem_op in mem_ops {
        for (addr, val) in &mem_op.ops {
            mem.insert(*addr, apply_mask1(val, &mem_op.mask));
        }
    }
    mem
}

fn apply_mask2(val: &i64, mask: &str) -> Vec<i64> {
    fn rec(val: i64, mask: &str, pos: usize, res: &mut Vec<i64>) {
        if pos == mask.len() {
            res.push(val);
            return;
        }
        let bit = 1 << (mask.len() - pos - 1);
        let c = mask.as_bytes()[pos] as char;
        if c == '0' {
            rec(val, mask, pos + 1, res);
        }
        if c == 'X' {
            rec(val & !bit, mask, pos + 1, res);
        }
        if c == '1' || c == 'X' {
            rec(val | bit, mask, pos + 1, res);
        }
    }
    let mut res: Vec<i64> = Vec::new();
    rec(*val, mask, 0, &mut res);
    res
}

fn eval2(mem_ops: &Vec<MemOp>) -> HashMap<i64, i64> {
    let mut mem = HashMap::new();
    for mem_op in mem_ops {
        for (addr, val) in &mem_op.ops {
            for masked_addr in apply_mask2(addr, &mem_op.mask) {
                mem.insert(masked_addr, *val);
            }
        }
    }
    mem
}

fn problem14() {
    let data = std::fs::read_to_string(&data_file(14)).unwrap();
    let ops = parse_ops(&data);
    println!("Answer 1: {:?}", eval1(&ops).values().sum::<i64>());
    println!("Answer 2: {:?}", eval2(&ops).values().sum::<i64>());
}

problem14();

Answer 1: 8471403462063
Answer 2: 2667858637669


In [108]:
// DAY15
fn eval(seed: &Vec<u64>, stop_k: u64) -> u64 {
    let mut positions = HashMap::new();
    let mut next_num = 0;
    let mut k: usize = 1;
    loop {
        let num = if k <= seed.len() {seed[k - 1]} else {next_num};
        if k as u64 == stop_k {
            return num;
        }
        match positions.get(&num) {
            Some(&x) => next_num = (k as u64) - x,
            _ => next_num = 0,
        };
        positions.insert(num, k as u64);
        k += 1;
    }
}

fn problem15() {
    let seed = std::fs::read_to_string(&data_file(15)).unwrap()
        .split(",")
        .map(|x| x.parse::<u64>().unwrap())
        .collect(); 
    println!("Answer 1: {}", eval(&seed, 2020));
    println!("Answer 2: {}", eval(&seed, 30000000));
}

problem15();

Answer 1: 412
Answer 2: 243


In [348]:
// DAY16
#[derive(Debug, PartialEq)]
struct Category {
    name: String,
    ranges: Vec<(i64, i64)>,
}

#[derive(Debug, PartialEq)]
struct Notes {
    categories: Vec<Category>,
    own_ticket: Vec<i64>,
    tickets: Vec<Vec<i64>>,
}

type Err = std::num::ParseIntError;

fn parse_notes(data: &str) -> Notes {
    fn parse_range(s: &str) -> Result<(i64, i64), Err> {
        let parts: Vec<&str> = s.split('-').collect();
        Ok((parts[0].parse()?, parts[1].parse()?))
    }
    
    fn parse_category(line: &str) -> Result<Category, Err> {
        let parts: Vec<&str> = line.split(": ").collect();
        let ranges = parts[1].split(" or ")
            .map(|s| parse_range(s).unwrap())
            .collect();
        Ok(Category{name: parts[0].to_owned(), ranges: ranges})
    }

    fn parse_ticket(line: &str) -> Result<Vec<i64>, Err> {
        line.split(',').map(|x| x.parse::<i64>()).collect()
    }
    
    let parts: Vec<Vec<&str>> = data.split("\n\n")
        .map(|s| s.trim().split('\n').collect())
        .collect();
    Notes {
        categories: parts[0].iter().map(|line| parse_category(line).unwrap()).collect(),
        own_ticket: parse_ticket(parts[1][1]).unwrap(),
        tickets: parts[2][1..].iter().map(|line| parse_ticket(line).unwrap()).collect()
    }
}

fn is_valid_category(category: &Category, val: &i64) -> bool {
    category.ranges.iter().any(|(a, b)| a <= val && val <= b)
}

fn get_invalid_entries(notes: &Notes) -> Vec<(usize, i64)> {
    let mut res = Vec::new();
    for (i, t) in notes.tickets.iter().enumerate() {
        let invalid = t.iter().filter(|n| 
            !notes.categories.iter().any(|cat| is_valid_category(cat, n)));
        for x in invalid {
            res.push((i, *x));
        }
    }
    res
}

fn get_valid_tickets(notes: &Notes, invalid_entries: &Vec<(usize, i64)>) -> Vec<Vec<i64>> {
    let invalid_tickets: HashSet<usize> = invalid_entries
        .into_iter()
        .map(|x| x.0)
        .collect();
    notes.tickets
        .iter()
        .enumerate()
        .filter_map(|(i, t)| 
            if invalid_tickets.contains(&i) {
                None
            } else {
                Some(t.clone())
            })
        .collect()
}

fn get_category_ordering(notes: &Notes) -> Vec<usize> {
    let n = notes.categories.len();
    let mut candidates = vec![vec![true; n]; n];
    let mut cands_left = n * n;
    
    // sift out invalid tickets
    for ticket in &notes.tickets {
        for (i, x) in ticket.iter().enumerate() {
            for j in 0..n {
                if !is_valid_category(&notes.categories[j], x) {
                    cands_left -= candidates[j][i] as usize;
                    candidates[j][i] = false;
                    
                }
            }
        }
    }
    
    // iteratively subtract single-element candidate sets from others
    while cands_left > n {
       for k in 0..n {
           let mut count = 0;
           let mut last = 0;
           for i in 0..n {
               if candidates[k][i] {
                   count += 1;
                   last = i;
                   if count > 1 {
                       break;
                   }
               }
           }
           if count == 1 {
               for i in 0..n {
                   if i != k {
                       cands_left -= candidates[i][last] as usize;
                       candidates[i][last] = false;
                       
                   }
               }
           }
        } 
    }
    
    let mut res = Vec::new();
    for i in 0..n {
        for j in 0..n {
            if candidates[j][i] {
                res.push(j);
            }
        }
    }
    res
}

fn part2(notes: &Notes) -> i64 {
    get_category_ordering(&notes)
        .iter()
        .enumerate()
        .filter_map(|(i, idx)|
            if notes.categories[*idx].name.starts_with("departure") {
                Some(notes.own_ticket[i])
            } else {
                None
            })
        .product()
}

fn problem16() {
    let notes = parse_notes(&fs::read_to_string(&data_file(16)).unwrap());
    let invalid_entries = get_invalid_entries(&notes);
    println!("Answer 1: {:?}", invalid_entries.iter().map(|(_, x)| x).sum::<i64>());
    let valid_tickets = get_valid_tickets(&notes, &invalid_entries);
    let valid_notes = Notes {
        categories: notes.categories,
        tickets: valid_tickets,
        own_ticket: notes.own_ticket,
    };
    println!("Answer 2: {:?}", part2(&valid_notes));
}

problem16();

Answer 1: 25916
Answer 2: 2564529489989


In [262]:
// DAY17
extern crate itertools;
use itertools::Itertools;

type Field = HashSet<Vec<i16>>;

fn count_neighbors(field: &Field, pos: &Vec<i16>) -> usize{
    pos.iter()
        .map(|x| (x - 1)..=(x + 1))
        .multi_cartesian_product()
        .filter(|p| p != pos && field.contains(p))
        .count()
}

fn get_bounds(field: &Field) -> Option<Vec<(i16, i16)>> {
    if field.len() == 0 {
        return  None;
    }
    let pos0 = field.iter().next().unwrap();
    let mut res: Vec<(i16, i16)> = pos0.iter().map(|x| (*x, *x)).collect();
    for pos in field.iter() {
        for (i, x) in pos.iter().enumerate() {
            res[i] = (*x.min(&res[i].0), *x.max(&res[i].1));
        }
    }
    Some(res)
}

fn step(field: &Field) -> Field {
    let mut res = Field::new();
    let bounds = get_bounds(&field);
    if bounds == None {
        return res;
    }
    let coord_it = bounds.unwrap()
        .iter()
        .map(|(a, b)| (a - 1)..=(b + 1))
        .multi_cartesian_product();
    for pos in coord_it {
        let n = count_neighbors(field, &pos);
        if field.contains(&pos) {
            if n == 2 || n == 3 {
                res.insert(pos);
            }
        } else {
            if n == 3 {
                res.insert(pos);
            }
        }
    }
    res
}

fn init_field(seed: &Vec<String>, ndim: usize) -> Field {
    let mut res = Field::new();
    for (y, line) in seed.iter().enumerate() {
        for (x, c) in line.as_bytes().iter().enumerate() {
            if *c as char == '#' {
                let mut key: Vec<i16> = Vec::new();
                key.resize(ndim, 0);
                key[0] = x as i16;
                key[1] = y as i16;
                res.insert(key);
            }
        }
    }
    res
}

const NUM_ITER: usize = 6;

fn run(seed: &Vec<String>, ndim: usize) -> usize {
    let mut field = init_field(&seed, ndim);
    for _ in 0..NUM_ITER {
        field = step(&field);
    }
    field.len()
}

fn problem17() {
    let seed = read_lines(&data_file(17)).unwrap();
    println!("Answer 1: {}", run(&seed, 3));
    println!("Answer 2: {}", run(&seed, 4));
}

problem17();

Answer 1: 223
Answer 2: 1884


In [190]:
// DAY18
#[derive(Debug)]
enum ArithOp {
    Add,
    Mul,
}

fn apply_op(op: Option<ArithOp>, a: Option<i64>, b: Option<i64>) -> Option<i64> {
    match op {
        Some(ArithOp::Add) => Some(a? + b?),
        Some(ArithOp::Mul) => Some(a? * b?),
        _ => if a == None {b} else {a}
    }
}

fn parse_num(exp: &str, pos: usize) -> (Option<i64>, usize) {
    let mut cpos = pos;
    let mut res = None;
    while cpos < exp.len() {
        let c = exp.as_bytes()[cpos] as char;
        if c < '0' || c > '9' {
            break;
        }
        let d = c as i64 - ('0' as i64);
        res = match res {
            Some(x) => Some(x * 10 + d),
            _ => Some(d)
        };
        cpos += 1;
    }
    (res, cpos)
}

fn eval_exp(exp: &str, pos: usize, add_precedence: bool) -> (Option<i64>, usize) {
    let mut res = None;
    let mut op =  None;
    let mut cpos = pos;
    let mut muls = Vec::new();
    while cpos < exp.len() {
        match exp.as_bytes()[cpos] as char {
            ')' => {
                cpos +=1;
                break;
            },
            '(' => {
                let (val, pos1) = eval_exp(exp, cpos + 1, add_precedence);
                res = apply_op(op, res, val);
                cpos = pos1;
                op = None;
            },
            ' ' => cpos += 1,
            '*' => {
                if add_precedence {
                    muls.push(res);
                    res = None;
                } else {
                    op = Some(ArithOp::Mul);
                }
                cpos += 1;
            },
            '+' => {
                op = Some(ArithOp::Add);
                cpos += 1;
            },
            _ => {
                let (val, pos1) = parse_num(exp, cpos);
                res = apply_op(op, res, val);
                op = None;
                cpos = pos1;
            }
        }
    }
    for mul in muls {
        res = Some(mul.unwrap() * res.unwrap());
    }
    (res, cpos)
}

fn problem18() {
    let lines = read_lines(&data_file(18)).unwrap();
    let res1: i64 = lines.iter().map(|line| eval_exp(line, 0, false).0.unwrap()).sum();
    println!("Answer 1: {}", res1);
    let res2: i64 = lines.iter().map(|line| eval_exp(line, 0, true).0.unwrap()).sum();
    println!("Answer 2: {}", res2);
}

problem18();

Answer 1: 45283905029161
Answer 2: 216975281211165


In [ ]:
// DAY19
use std::iter;

#[derive(Debug)]
enum GrammarRule {
    Term(String),
    NonTerm(Vec<String>),
}

type GrammarRuleSet = HashMap<String, Vec<GrammarRule>>;

fn parse_grammar_rule(text: &str) -> Vec<GrammarRule> {
   let mut res = Vec::new();
    for x in text.split("|").map(|s| s.trim()) {
        if x.starts_with("\"") {
            res.push(GrammarRule::Term(x[1..x.len() - 1].to_string()));
        } else {
            res.push(GrammarRule::NonTerm(x.split(" ").map(|s| s.to_string()).collect()))
        }
    }
    res
}

fn parse_grammar_rules(text: &str) -> (GrammarRuleSet, Vec<String>) {
    let chunks: Vec<Vec<&str>> = text
        .split("\n\n")
        .map(|s| s.trim().split("\n").collect())
        .collect();
    let mut rules = HashMap::new();
    for line in &chunks[0] {
        let parts: Vec<&str> = line.split(": ").collect();
        rules.insert(parts[0].to_string(), parse_grammar_rule(parts[1]));
    }
    (rules, chunks[1].iter().map(|s| s.to_string()).collect())
}

fn is_valid(rules: &GrammarRuleSet, line: &str) -> bool {
    fn match_seq(rules: &GrammarRuleSet, seq: &[String], line: &str, pos: usize) -> Vec<usize> {
        let mut res = Vec::new();
        if seq.is_empty() {
            res.push(pos);
        } else {
            for cpos in match_rule(rules, &seq[0], line, pos) {
                res.extend(match_seq(rules, &seq[1..], line, cpos));
            }
        }
        res
    }
        
    fn match_rule(rules: &GrammarRuleSet, rule_name: &str, line: &str, pos: usize) -> Vec<usize> {
        let mut res = Vec::new();
        for option in &rules[rule_name] {
            match option {
                GrammarRule::Term(s) if line[pos..].starts_with(s) => 
                    res.push(pos + s.len()),
                GrammarRule::NonTerm(opts) => 
                    res.extend(match_seq(rules, &opts[0..], line, pos)),
                _ => ()
            };
        }
        res
    }
    
    let res = match_rule(rules, "0", line, 0);
    if res.len() == 0 {
        false
    } else {
        res[0] == line.len()
    }
}

fn patch_rules(rules: &mut GrammarRuleSet) {
    rules.insert("8".to_string(), parse_grammar_rule("42 | 42 8"));
    rules.insert("11".to_string(), parse_grammar_rule("42 31 | 42 11 31"));
}

fn problem19() {
    let (mut rules, strings) = parse_grammar_rules(&fs::read_to_string(&data_file(19)).unwrap());
    let res1: u32 = strings.iter().map(|s| is_valid(&rules, s) as u32).sum();
    println!("Answer 1: {}", res1);
    
    patch_rules(&mut rules);
    let res2: u32 = strings.iter().map(|s| is_valid(&rules, s) as u32).sum();
    println!("Answer 2: {}", res2);
}

problem19();

In [147]:
// DAY20
use std::fs;
use std::fmt;

type TileId = u64;

#[derive(Debug)]
struct Tile {
    pt: Vec<Vec<char>>,
    id: TileId,
}

#[derive(Eq, PartialEq, Hash, Clone, Debug)]
enum Dir {
    N,
    E,
    S,
    W,
}

impl Dir {
    fn opposite(&self) -> Dir {
        match self {
            Dir::N => Dir::S,
            Dir::E => Dir::W,
            Dir::S => Dir::N,
            Dir::W => Dir::E,
        }
    }
    fn opposite_idx(&self) -> usize {
        Dir::ENUM
            .iter()
            .position(|d| *d == self.opposite())
            .unwrap()
    }
    const ENUM: [Dir; 4] = [Dir::N, Dir::E, Dir::S, Dir::W];
    const OFFS: [(i64, i64); 4] = [(0, -1), (1, 0), (0, 1), (-1, 0)];
}

#[derive(Debug, Clone)]
struct Transform {
    rotation: Dir,
    flip_x: bool,
    flip_y: bool,
}

type TileSet = Vec<Tile>;

#[derive(Debug, Clone)]
struct TileInstance {
    tile_idx: usize,
    tm: Transform,
    edges: [String; 4],
}

#[derive(Eq, PartialEq, Hash, Clone, Debug)]
struct TileMappingKey {
    dir: Dir,
    edge: String,
}

type TileMapping = HashMap<TileMappingKey, Vec<TileInstance>>;
type TileLayout = HashMap<(i64, i64), TileInstance>;

impl Tile {
    fn parse(text: &str, has_header: bool) -> Option<Tile> {
        let lines: Vec<&str> = text.split("\n").map(|s| s.trim()).collect();
        if lines.len() < 2 {
            return None;
        }
        let (id, rows) = if has_header {
            let header = lines[0].split(" ").collect::<Vec<&str>>()[1];
            (header[..header.len() - 1].parse().ok()?, lines[1..].to_vec())
        } else {
            (0, lines[0..].to_vec())
        };
        Some(Tile {
            id: id,
            pt: rows.iter().map(|s| s.chars().collect()).collect(),
        })
    }
    fn empty(w: usize, h: usize) -> Tile {
        Tile {
            id: 0,
            pt: vec![vec!['.'; w]; h],
        }
    }
    fn count(&self, ch: char) -> usize {
        self.pt.iter().flat_map(|row| row.iter().map(|c| (*c == ch) as usize)).sum()
    }
    fn get_extents(&self) -> (usize, usize) {
        (self.pt[0].len(), self.pt.len())
    }
    fn get_edge(&self, dir: &Dir) -> String {
        match dir {
            Dir::N => self.pt[0].iter().collect(),
            Dir::E => self.pt.iter().map(|s| *s.last().unwrap()).collect(),
            Dir::S => self.pt.last().unwrap().iter().collect(),
            Dir::W => self.pt.iter().map(|s| s[0]).collect(),
        }
    }
    fn get_edges(&self) -> [String; 4] {
        [
            self.get_edge(&Dir::N),
            self.get_edge(&Dir::E),
            self.get_edge(&Dir::S),
            self.get_edge(&Dir::W),
        ]
    }
    fn transform(&self, tm: &Transform) -> Tile {
        let n = self.pt.len();
        let mut res = Tile::empty(n, n);
        if n == 0 {
            return res;
        }
        assert!(n == self.pt[0].len());
        for j in 0..n {
            for i in 0..n {
                let (mut x, mut y) = (i, j);
                if tm.flip_x {
                    x = n - x - 1;
                }
                if tm.flip_y {
                    y = n - y - 1;
                }
                let (rx, ry) = match tm.rotation {
                    Dir::N => (x, y),
                    Dir::E => (y, n - x - 1),
                    Dir::S => (n - x - 1, n - y - 1),
                    Dir::W => (n - y - 1, x),
                };
                res.pt[j][i] = self.pt[ry][rx];
            }
        }
        res
    }
    fn blit(&mut self, tile: &Tile, x: usize, y: usize, border: usize) {
        let n = tile.pt.len();
        for j in 0..(n - border * 2) {
            for i in 0..(n - border * 2) {
                self.pt[y + j][x + i] = tile.pt[j + border][i + border];
            }
        }
    }
    fn matches(&self, tile: &Tile, x: usize, y: usize) -> bool {
        for (j, row) in tile.pt.iter().enumerate() {
            for (i, c) in row.iter().enumerate() {
                if *c == '#' && self.pt[y + j][x + i] != *c {
                    return false;
                }
            }
        }
        true
    }
    fn find_matches(&self, pattern: &Tile) -> Vec<(i64, i64)> {
        let mut res = Vec::new();
        let (w, h) = self.get_extents();
        let (tw, th) = pattern.get_extents();
        for j in 0..(h - th + 1) {
            for i in 0..(w - tw + 1) {
                if self.matches(pattern, i, j) {
                    res.push((i as i64, j as i64));
                }
            }
        }
        res
    }
}

impl fmt::Display for Tile {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        for row in &self.pt {
            for c in row.iter() {
                write!(f, "{}", c);
            }
            write!(f, "\n");
        }
        Ok(())
    }
}

fn create_mapping(tiles: &TileSet) -> TileMapping {
    let mut res = TileMapping::new();
    for (i, tile) in tiles.iter().enumerate() {
        for flip_x in &[false, true] {
            for flip_y in &[false, true] {
                if *flip_x && *flip_y {
                    continue;
                }
                for rotation in &Dir::ENUM {
                    let tm = Transform {
                        rotation: rotation.clone(),
                        flip_x: *flip_x,
                        flip_y: *flip_y,
                    };
                    let tile_transformed = tile.transform(&tm);
                    for dir in &Dir::ENUM {
                        let edge = tile_transformed.get_edge(dir);
                        let key = TileMappingKey {
                            dir: dir.clone(),
                            edge: edge,
                        };
                        let tile_instance = TileInstance {
                            tile_idx: i,
                            tm: tm.clone(),
                            edges: tile_transformed.get_edges(),
                        };
                        res.entry(key).or_insert(Vec::new()).push(tile_instance);
                    }
                }
            }
        }
    }
    res
}

fn find_corners(tiles: &TileSet, mapping: &TileMapping) -> Vec<usize> {
    let mut res: Vec<usize> = Vec::new();
    for (i, tile) in tiles.iter().enumerate() {
        let mut num_unique_edges = 0;
        for dir in &Dir::ENUM {
            let key = TileMappingKey {
                dir: dir.opposite(),
                edge: tile.get_edge(dir),
            };
            num_unique_edges += (match mapping.get(&key) {
                Some(matches) => matches.iter().map(|t| (i != t.tile_idx) as u32).sum(),
                _ => 0,
            } == 0) as u32;
        }
        if num_unique_edges == 2 {
            res.push(i);
        }
    }
    res
}

fn get_layout(tiles: &TileSet, mapping: &TileMapping) -> TileLayout {
    fn rec(
        tile_idx: usize,
        tm: &Transform,
        pos: (i64, i64),
        tiles: &TileSet,
        mapping: &TileMapping,
        used: &mut Vec<bool>,
        res: &mut TileLayout,
    ) -> bool {
        if used[tile_idx] {
            return false;
        }
        // find if existing neighbors fit with the transformed tile's edges
        let tile_transformed = tiles[tile_idx].transform(&tm);
        let edges = tile_transformed.get_edges();
        for (i, edge) in edges.iter().enumerate() {
            let offs = Dir::OFFS[i];
            let pos1 = (pos.0 + offs.0, pos.1 + offs.1);
            match res.get(&pos1) {
                Some(t) => {
                    if t.edges[Dir::ENUM[i].opposite_idx()] != *edge {
                        return false;
                    }
                }
                _ => (),
            }
        }
        used[tile_idx] = true;
        res.insert(
            pos,
            TileInstance {
                tile_idx: tile_idx,
                tm: tm.clone(),
                edges: edges.clone(),
            },
        );
        if res.len() == tiles.len() {
            return true;
        }
        // recurse into neighbors
        let mut found_fit = false;
        for (i, edge) in edges.iter().enumerate() {
            let offs = Dir::OFFS[i];
            let pos1 = (pos.0 + offs.0, pos.1 + offs.1);
            if res.contains_key(&pos1) {
                // neighbor position is already occupied
                continue;
            }
            let key = TileMappingKey {
                dir: Dir::ENUM[i].opposite(),
                edge: edge.clone(),
            };
            match mapping.get(&key) {
                Some(instances) => {
                    if instances
                        .iter()
                        .any(|inst| rec(inst.tile_idx, &inst.tm, pos1, tiles, mapping, used, res))
                    {
                        found_fit = true;
                        break;
                    }
                }
                _ => (),
            }
        }
        if !found_fit {
            used[tile_idx] = false;
            res.remove(&pos);
        }
        found_fit
    }
    let mut res = TileLayout::new();
    let mut used = vec![false; tiles.len()];
    let corners = find_corners(tiles, mapping);
    let start_idx = corners[0];
    rec(
        start_idx,
        &Transform {
            rotation: Dir::N,
            flip_x: false,
            flip_y: false,
        },
        (0, 0),
        tiles,
        mapping,
        &mut used,
        &mut res,
    );
    res
}

fn blit_layout(tiles: &TileSet, layout: &TileLayout) -> Tile {
    let minx = layout.keys().map(|x| x.0).min().unwrap();
    let maxx = layout.keys().map(|x| x.0).max().unwrap();
    let miny = layout.keys().map(|x| x.1).min().unwrap();
    let maxy = layout.keys().map(|x| x.1).max().unwrap();
    let n = tiles[0].pt.len() - 2;
    let mut res = Tile::empty(n * (maxx - minx + 1) as usize, n * (maxy - miny + 1) as  usize);
    for (pos, instance) in layout {
        let tile_transformed = tiles[instance.tile_idx].transform(&instance.tm);
        res.blit(&tile_transformed, n * (pos.0 - minx) as usize, n * (pos.1 - miny) as usize, 1);
    }
    res
}

fn find_pattern(img: &Tile, pattern: &Tile) -> Vec<(i64, i64)>{
    for flip_x in &[false, true] {
        for flip_y in &[false, true] {
            if *flip_x && *flip_y {
                continue;
            }
            for rotation in &Dir::ENUM {
                let tm = Transform {
                    rotation: rotation.clone(),
                    flip_x: *flip_x,
                    flip_y: *flip_y,
                };
                let img1 = img.transform(&tm);
                let matches = img1.find_matches(pattern);
                if  matches.len() > 0  {
                    return matches;
                }
            }
        }
    }
    Vec::new()
}

fn problem20() {
    let tiles: TileSet = fs::read_to_string(&data_file(20))
        .unwrap()
        .split("\n\n")
        .filter_map(|s| Tile::parse(s,true))
        .collect();
    let mapping = create_mapping(&tiles);
    let corners = find_corners(&tiles, &mapping);
    let res1: u64 = corners.iter().map(|i| tiles[*i].id).product();
    println!("Answer 1: {}", res1);
    
    let pattern = Tile::parse(&fs::read_to_string(&extra_file(20, "pattern")).unwrap(), false).unwrap();
    let layout = get_layout(&tiles, &mapping);
    let img = blit_layout(&tiles, &layout);
    let m = find_pattern(&img, &pattern);
    println!("Answer 2: {}", img.count('#') - m.len() * pattern.count('#'));
}

problem20();

Answer 1: 63187742854073
Answer 2: 2152


In [223]:
// DAY21
type Rule = (HashSet<String>, HashSet<String>);
type Mapping = HashMap<String, String>;

fn parse_rule(line: &str) -> Rule {
    let mut parts = line.split(" (contains ");
    let c = parts.next().unwrap().split(" ").map(|s| s.to_string()).collect();
    let a = parts.next().unwrap().trim_matches(')').split(", ").map(|s| s.to_string()).collect();
    (c, a)
}

fn get_mapping(rules: &Vec<Rule>) -> Mapping {
    fn first(set: &HashSet<String>) -> String {
        set.iter().next().unwrap().clone()
    }
    let mut res = Mapping::new();
    for (i, (c, a)) in rules.iter().enumerate() {
        if c.len() == 1 && a.len() == 1 {
            res.insert(first(c), first(a));
            continue;
        }
        let mut cxc = c.clone();
        let mut cxa = a.clone();
        for j in (i + 1)..rules.len() {
            let (c1, a1) = &rules[j];
            let xa: HashSet<String> = cxa.intersection(&a1).map(|s| s.to_string()).collect();
            if !xa.is_empty() {
                cxc = cxc.intersection(&c1).map(|s| s.to_string()).collect();
                cxa = xa;
            }
            if cxc.len() == 1 && cxa.len() == 1 {
                res.insert(first(&cxc), first(&cxa));
                break;
            } 
        }
    }
    res
}

fn trim_rules(rules: &mut Vec<Rule>) -> Mapping {
    let mut res = Mapping::new();
    loop {
        let mapping = get_mapping(rules);
        if mapping.is_empty() {
            return res;
        }
        for (c, a) in mapping {
            res.insert(c.clone(), a.clone());
            for i in 0..rules.len() {
                rules[i].0.remove(&c);
                rules[i].1.remove(&a);
            }
        }
    }
}
        
fn problem21() {
    let mut rules: Vec<Rule> = read_lines(&data_file(21)).unwrap()
        .iter()
        .map(|s| parse_rule(s))
        .collect();
    
    let mapping = trim_rules(&mut rules);
    let res1: usize = rules.iter().map(|r| r.0.len()).sum();
    println!("Answer 1: {}", res1);
    
    let mut m: Vec<(&String, &String)> = mapping.iter().collect();
    m.sort_by(|a, b| a.1.partial_cmp(b.1).unwrap());
    println!("Answer 2: {}", m.iter().map(|x| x.0.clone()).collect::<Vec<String>>().join(","));
}

problem21();

Answer 1: 2374
Answer 2: fbtqkzc,jbbsjh,cpttmnv,ccrbr,tdmqcl,vnjxjg,nlph,mzqjxq


In [154]:
// DAY22
use std::fs;
use std::collections::VecDeque;
use std::collections::hash_map::DefaultHasher;
use std::hash::{Hash, Hasher};

type Deck = VecDeque<usize>;

fn parse_deck(text: &str) -> Deck {
    text.split("\n")
        .skip(1)
        .filter_map(|s| s.parse::<usize>().ok())
        .collect()
}

fn step(da: &mut Deck, db: &mut Deck, recurse: bool) -> Option<usize> {
    if da.len() == 0 {
        return Some(1);
    }  else if db.len() == 0 {
        return Some(0);
    }
    let (a, b) = (da.pop_front().unwrap(), db.pop_front().unwrap());
    let a_wins = if recurse && a <= da.len() && b <= db.len() {
        let mut da1 = da.iter().take(a).map(|x| *x).collect();
        let mut db1 = db.iter().take(b).map(|x| *x).collect();
        game(&mut da1, &mut db1, true) == 0
    } else {
        a > b
    };
    
    if a_wins {
        da.push_back(a);
        da.push_back(b);
    } else {
        db.push_back(b);
        db.push_back(a);
    }
    None
}

fn game(da: &mut Deck, db: &mut Deck, recurse: bool) -> usize {
    let (mut visited_a, mut visited_b) = (HashSet::new(), HashSet::new());
    loop {
        let (mut hasher1, mut hasher2) = (DefaultHasher::new(), DefaultHasher::new());
        da.hash(&mut hasher1);
        db.hash(&mut hasher2);
        let (h1, h2) = (hasher1.finish(), hasher2.finish());
        if visited_a.contains(&h1) || visited_b.contains(&h2) {
            return 0;
        }
        visited_a.insert(h1);
        visited_b.insert(h2);
        
        match step(da, db, recurse) {
            Some(x) => return x,
            None => ()
        }
    }
}

fn get_score(deck: &Deck ) -> usize {
    deck.iter().enumerate().map(|(i, x)| x * (deck.len() - i)).sum()
}

fn problem22() {
    let mut decks: Vec<Deck> = fs::read_to_string(&data_file(22))
        .unwrap()
        .split("\n\n")
        .map(|s| parse_deck(s))
        .collect();
    
    let (mut da1, mut db1) = (decks[0].clone(), decks[1].clone());
    let winner1 = game(&mut da1, &mut db1, false);
    println!("Answer 1: {}", get_score(if winner1 == 0 {&da1} else {&db1}));
    
    let (mut da2, mut db2) = (decks[0].clone(), decks[1].clone());
    let winner2 = game(&mut da2, &mut db2, true);
    println!("Answer 2: {}", get_score(if winner2 == 0 {&da2} else {&db2}));    
}

problem22();

Answer 1: 31809
Answer 2: 32835


In [69]:
// DAY23
use std::fs;
use std::fmt;
use std::iter::FromIterator;

#[derive(Debug)]
struct Node {
    val: usize,
    next: usize,
}

#[derive(Debug)]
struct CircularList {
    nodes: Vec<Node>,
    current: usize,
    lookup: HashMap<usize, usize>,
    min_val: usize,
    max_val: usize,
}

impl FromIterator<usize> for CircularList {
    fn from_iter<I: IntoIterator<Item=usize>>(iter: I) -> Self {
        let mut lookup = HashMap::new();
        let mut nodes = Vec::new();
        let mut i = 0;
        let (mut min_val, mut max_val) = (usize::MAX, 0);
        for val in iter {
            nodes.push(Node {val: val, next: i + 1});
            lookup.insert(val, i);
            i += 1;
            min_val = val.min(min_val);
            max_val = val.max(max_val);
        }
        let n = nodes.len();
        nodes[n - 1].next = 0;
        CircularList {
            nodes: nodes,
            current: 0,
            lookup: lookup,
            min_val: min_val,
            max_val: max_val,
        }
    }
}

impl fmt::Display for CircularList {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        let start_node = self.current;
        write!(f, "({}) ", self.nodes[start_node].val);
        let mut node = self.nodes[start_node].next;
        while node != start_node {
            write!(f, "{} ", self.nodes[node].val);
            node = self.nodes[node].next;
        }
        Ok(())
    }
}

impl CircularList {
    fn get_values(&self, num_values: usize, start_val: usize) -> Vec<usize> {
        let mut i = 0;
        let start_node = self.lookup[&start_val];
        let mut res = Vec::new();
        let mut node = self.nodes[start_node].next;
        loop {
            if node == start_node || i == num_values {
                return res;
            }
            res.push(self.nodes[node].val);
            node = self.nodes[node].next;
            i += 1;
        }
    }

    fn step(&mut self) {
        let mut to_move = vec![0; 3];
        let mut to_move_values = vec![0; 3];
        let mut node = self.nodes[self.current].next;
        for i in 0..3 {
            to_move[i] = node;
            to_move_values[i] = self.nodes[node].val;
            node = self.nodes[node].next;
        }
        self.nodes[self.current].next = node;


        let mut dest_val = self.nodes[self.current].val - 1;
        while to_move_values.contains(&dest_val) || 
            !self.lookup.contains_key(&dest_val) {
            if dest_val < self.min_val {
                dest_val = self.max_val;
            } else {
                dest_val -= 1;
            }
        }
        let dest_node = self.lookup[&dest_val];

        let dnext = self.nodes[dest_node].next;
        self.nodes[dest_node].next = to_move[0];
        self.nodes[to_move[to_move.len() - 1]].next = dnext;    
        self.current = self.nodes[self.current].next;
    }
}

fn eval_game(labels: &mut CircularList, steps: usize) {
    for i in 0..steps {
        if (i + 1) % 1000000 == 0 {
            println!("Step {}", i);
        }
        labels.step();
    }
}

fn problem23() {
    let now = Instant::now();
    let seed: Vec<usize> = fs::read_to_string(&data_file(23)).unwrap()
        .split("")
        .filter(|s| !s.is_empty())
        .map(|x| x.parse::<usize>().unwrap())
        .collect();
    
    let mut labels1 = CircularList::from_iter(seed.clone().into_iter());
    eval_game(&mut labels1, 100);
    let res1 = labels1.get_values(usize::MAX, 1).iter().map(|x| x.to_string()).collect::<Vec<String>>().join("");
    println!("Answer 1: {}", res1);
    
    let max_val = seed.iter().max().unwrap();
    let extend_range = (max_val + 1)..(1000000 + max_val + 1 - seed.len());
    let mut labels2 = CircularList::from_iter(seed.into_iter().chain(extend_range));
    eval_game(&mut labels2, 10000000);
    let res2: usize = labels2.get_values(2, 1).iter().product();
    println!("Answer 2: {}", res2);
    
    let elapsed = now.elapsed();
    println!("Elapsed: {:.2?}", elapsed);
}

problem23();

Answer 1: 24798635
Step 999999
Step 1999999
Step 2999999
Step 3999999
Step 4999999
Step 5999999
Step 6999999
Step 7999999
Step 8999999
Step 9999999
Answer 2: 12757828710
Elapsed: 5.22s


In [52]:
// DAY24
use std::slice::Iter;

#[derive(Debug)]
enum HexDir {E, W, NW, SW, NE, SE}
type HexCoord = (i64, i64);

fn parse_path(line: &str) -> Vec<HexDir> {
    let mut it = line.chars();
    let mut res = Vec::new();
    loop {
        let dir = match it.next() {
            Some('e') => HexDir::E,
            Some('w') => HexDir::W,
            Some(x) => {
                match (x, it.next()) {
                    ('n', Some('w')) => HexDir::NW,
                    ('n', Some('e')) => HexDir::NE,
                    ('s', Some('w')) => HexDir::SW,
                    ('s', Some('e')) => HexDir::SE,
                    _ => return res
                }
            },
            _ => return res
        };
        res.push(dir);
    }
}

fn eval_step((x, y): &HexCoord, step: &HexDir) -> HexCoord {
    let dy = match step {
        HexDir::SW | HexDir::SE => 1,
        HexDir::NW | HexDir::NE => -1,
        _ =>  0
    };
    let dx = match step {
        HexDir::E => 1,
        HexDir::W => -1,
        HexDir::SW | HexDir::NW if y % 2 == 0 => -1,
        HexDir::SE | HexDir::NE if y % 2 != 0 => 1,
        _ => 0,
    };
    (x + dx, y + dy)
}

fn eval_path(steps: &Vec<HexDir>) -> HexCoord {
    steps.iter().fold((0, 0), |pos, step| eval_step(&pos, step))
}

fn eval_paths(paths: &Vec<Vec<HexDir>>) -> HashSet<HexCoord> {
    let mut res = HashSet::new();
    for path in paths {
        let pos = eval_path(path);
        if res.contains(&pos) {
            res.remove(&pos);
        } else{
            res.insert(pos);
        }
    }
    res
}

fn step_day(tiles: &HashSet<HexCoord>) -> HashSet<HexCoord> {
    let minx = tiles.iter().map(|t| t.0).min().unwrap();
    let maxx = tiles.iter().map(|t| t.0).max().unwrap();
    let miny = tiles.iter().map(|t| t.1).min().unwrap();
    let maxy = tiles.iter().map(|t| t.1).max().unwrap();
    
    let mut res = HashSet::new();
    for x in (minx - 1)..=(maxx + 1) {
        for y in (miny - 1)..=(maxy + 1) {
            let pos = (x, y);
            let num_neighbors: u32 = 
                [HexDir::E, HexDir::W, HexDir::NW, HexDir::SW, HexDir::NE, HexDir::SE]
                .iter()
                .map(|dir| tiles.contains(&eval_step(&pos, dir)) as u32)
                .sum();
            if tiles.contains(&pos) {
                if num_neighbors == 0 || num_neighbors > 2 {
                } else {
                    res.insert(pos);
                }
            } else if num_neighbors == 2 {
               res.insert(pos); 
            }
        }
    }
    res
}

fn problem24() {
   let paths: Vec<Vec<HexDir>> = read_lines(&data_file(24))
        .unwrap()
        .iter()
        .map(|s| parse_path(s))
        .collect();
    
    let mut tiles = eval_paths(&paths);
    println!("Answer 1: {}", tiles.len());
    
    for i in 0..100 {
        tiles = step_day(&tiles);
    }
    println!("Answer 2: {}", tiles.len());
}

problem24();

Answer 1: 469
Answer 2: 4353


In [236]:
// DAY25
const FACTOR: usize = 20201227;
const DEFAULT_SUBJ_NUMBER: usize = 7;

fn tm_step(val: usize, subj_number: usize) -> usize {
    (val * subj_number) % FACTOR
}

fn transform(subj_number: usize, loop_size: usize) -> usize {
    (0..loop_size).fold(1, |val, _| tm_step(val, subj_number))
}

fn find_loop_len(pub_key: usize, subj_number: usize) -> usize {
    let mut val = 1;
    let mut i = 0;
    while val != pub_key {
        val = tm_step(val, subj_number);
        i += 1
    }
    return i;
}

fn problem25() {
    let pub_keys: Vec<usize> = read_lines(&data_file(25))
        .unwrap()
        .iter()
        .map(|s| s.parse().unwrap())
        .collect();
    
    let loop_sizes: Vec<usize> = pub_keys
        .iter()
        .map(|x| find_loop_len(*x, DEFAULT_SUBJ_NUMBER))
        .collect();
    let res = transform(pub_keys[0], loop_sizes[1]);
    println!("Answer: {}", res);
}

problem25();

Answer: 9177528
